In [1]:
import pandas as pd
import chardet

In [2]:
with open("raw_data/trade-register.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result["encoding"])

ISO-8859-1


In [3]:
df = pd.read_csv('raw_data/trade-register.csv', encoding=result["encoding"])


In [4]:
# 10 most valuable per unit items
df.sort_values('SIPRI TIV per unit', ascending=False).head(10)


,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
10670,India,Russia,2004.0,NaN,1.0,NaN,Gorshkov,aircraft carrier,1.0,NaN,2013,New,$2.3 b deal (originally $625-750 m deal but in...,1250.00,1250.00,1250.00
10050,India,Russia,2019.0,NaN,1.0,NaN,Project-971I,nuclear submarine,0.0,NaN,NaN,New,$3 b deal for 10-year lease; Indian designatio...,1000.00,1000.00,0.00
10540,India,Russia,2004.0,?,1.0,NaN,Project-971I,nuclear submarine,1.0,NaN,2012,New,$650 m 10-year lease (returned to Russia 2021)...,1000.00,1000.00,1000.00
4643,China,Ukraine,1998.0,NaN,1.0,NaN,Kuznetsov,aircraft carrier,1.0,NaN,2012,Second hand,Second-hand (production stopped 1992 with end ...,631.60,631.60,631.60
4782,China,Russia,2002.0,NaN,2.0,NaN,Project-956/Sovremenny,destroyer,2.0,NaN,2005; 2006,New,$1-1.5 b deal; Type-956EM version; option on 2...,500.00,1000.00,1000.00
4780,China,Russia,1998.0,NaN,2.0,NaN,Project-956/Sovremenny,destroyer,2.0,NaN,1999; 2000,New,$667 m deal; Project-956E version; originally ...,500.00,1000.00,1000.00
1292,Australia,Spain,2007.0,NaN,3.0,NaN,Hobart,destroyer,3.0,NaN,2017; 2018; 2020,New,AUD8.5-9.7 b ($7.2-8.2 b) 'Project Sea-4000' o...,499.00,1497.00,1497.00
7328,Egypt,Italy,2020.0,NaN,2.0,NaN,FREMM,frigate,2.0,NaN,2020; 2021,New,Originally produced for Italy but sold to Egypt,483.00,966.00,966.00
16093,Morocco,France,2008.0,NaN,1.0,NaN,FREMM,frigate,1.0,NaN,2014,New,EUR470 m deal; Moroccan designation Mohammed VI,482.95,482.95,482.95
10771,Indonesia,Italy,2021.0,?,6.0,NaN,FREMM,frigate,0.0,NaN,NaN,New,Selected 2021 but not yet ordered by end-2023,482.95,2897.70,0.00


In [5]:
len(df['Weapon description'].unique())

222

## Categorize weapon descriptions
Make a high level category and a sub-category for each weapon description 

In [6]:
category_mapping = {
    'Aircraft': [
        'fighter aircraft', 'bomber aircraft', 'transport aircraft', 'reconnaissance aircraft', 
        'trainer aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'trainer/combat aircraft', 
        'AGS aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'airship', 'target tow aircraft', 'light aircraft', 'ground attack aircraft',
        'ASW aircraft', 'AEW aircraft', 'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'transport ac/helicopter', 'transport/trainer aircraft', 'airship', 
        'light transport aircraft', 'heavy transport aircraft', 'tanker/transport aircraft', 'trainer/light aircraft', 'light/trainer aircraft'
    ],
    'Helicopters': [
        'combat helicopter', 'transport helicopter', 'ASW helicopter', 'light helicopter', 
        'AEW helicopter', 'anti-ship helicopter', 'SIGINT helicopter', 'helicopter', 'ASW Helicopter'
    ],
    'Unmanned Vehicles': [
        'UAV', 'armed UAV', 'reconnaissance AV', 'maritime patrol UAV', 'light aircraft/UAV', 'loitering munition' 
    ],
    'Naval Vessels': [
        'frigate', 'destroyer', 'corvette', 'patrol craft', 'submarine', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'gunboat', 'landing ship', 'salvage ship', 
        'survey ship', 'cruiser', 'aircraft carrier', 'minelayer', 'replenishment tanker', 
        'corvette/minesweeper', 'OPV/transport ship', 'support/landing ship', 'landing/patrol craft',
        'transport ship', 'FAC (Fast Attack Craft)', 'minesweeper', 'landing craft', 'tug', 'nuclear submarine', 'OPV/tug',
        'frigate/landing ship', 'SSB (Ballistic Missile Submarine)', 'SIGINT ship', 'patrol craft/transport craft', 'transport craft',
        'support craft', 'cargo craft', 'midget submarine', 'icebreaker/OPV', 'FAC', 'SSB'
    ],
    'Ground Vehicles/Artillery': [
        'tank', 'light tank', 'armoured car', 'APC', 'IFV', 'AFSV', 'APV', 'AMV', 'ARV', 'AEV/ARV', 
        'armoured bridgelayer', 'training tank', 'tank turret', 'self-propelled gun chassis', 'tank chassis',
        'APC/APV', 'APC/IFV', 'APC/CP', 'IFV/AFSV', 'IFV/APC turret', 'APC turret', 'IFV turret', 'self-propelled AD system',
        'self-propelled gun', 'self-propelled MRL', 'towed gun', 'self-propelled AA gun', 'coastal defence system', 'AA gun (Anti-Aircraft Gun)',
        'towed MRL', 'ALV (Armored Logistics Vehicle)', 'naval gun', 'tank destroyer', 'mortar', 'self-propelled mortar', 'anti-aircraft gun',
        'SPG turret', 'ASW mortar', 'AEV (Armored Engineering Vehicle)', 'helicopter turret', 'anti-tank AV turret', 'self-propelled AA gun turret',
        'apc (Armored Personnel Carrier)', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun', 'ALV', 'AEV', 'apc'
    ],
    'Missiles/Rockets/Bombs': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile',
        'SSM', 'SSM launcher', 'SSM TEL', 'anti-ship missile', 'anti-radar missile', 'anti-tank missile', 
        'guided bomb', 'guided rocket', 'guided rocket/SSM', 'guided rocket/ASM', 'SSM/ASM', 'anti-ship missile/SSM',
        'anti-ship/land-attack missile', 'land-attack missile', 'anti-tank missile/ASM', 'guided glide bomb',
        'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'ASW torpedo', 'ASW MRL', 
        'anti-ship missile/ASM', 'BVRAAM', 'BVRAAM/SAM', 'SRAAM', 'ASM', 'ABM/SAM system', 'naval MRL', 'ASM (Anti-Ship Missile)',
        'guided shell', 'ASW missile', 'anti-ship/anti-radar missile', 'naval mine/torpedo', 'ABM system', 'ASW rocket launcher',
        'anti-ship missile/ASM/SAM', 'SLBM (Submarine-Launched Ballistic Missile)', 'ASM', 'SLBM', 'anti-radar missile/ASM'
    ],
    'Radar/Detection Systems': [
        'air search radar', 'AGS radar', 'sea search radar', 'multi-function radar', 'air search system', 
        'fire control radar', 'air/sea search radar', 'artillery locating radar', 'ground surv radar', 
        'ground/sea search radar', 'AEW radar', 'MP aircraft radar', 'aircraft EO system', 
        'aircraft EO/radar system', 'air/ground surv radar', 'SAM system radar', 'multi-role radar', 'radar',
        'combat aircraft radar', 'combat heli radar', 'SONAR', 'submarine sonar', 'ASW sonar', 'SONAR system',
        'aircraft recce system', 'height-finding radar', 'ASW sonar', 'SONAR', 'recce satellite', 'EO search/fire control', 'EO system',
        'AGS/MP aircraft radar', 'Naval EO system', 'AGS/SIGINT system', 'surveillance satellite'
    ],
    'Specialized Equipment': [
        'aircraft engine', 'turbofan', 'turbojet', 'gas turbine', 'vehicle engine', 'AIP engine', 
        'air refuel system', 'maritime patrol aircraft', 'SIGINT system', 
        'SIGINT aircraft', 'AALS', 'VEHICLE ENGINE', 'turbojet', 'nuclear reactor', 'ship engine', 'turboprop'
    ],
    'Other': [
        'SSB' , 'training equipment'
    ]
}

# Expanded sub-category mappings
sub_category_mapping = {
    # Aircraft Sub-categories
    'Aircraft': {
        'Fighter/Combat Aircraft': ['fighter aircraft', 'trainer/combat aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'ground attack aircraft', 'ASW aircraft'],
        'Bomber/Heavy Aircraft': ['bomber aircraft', 'heavy transport aircraft'],
        'Reconnaissance/Surveillance Aircraft': ['reconnaissance aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'AGS aircraft', 'light aircraft', 'AEW aircraft',
                                                'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'airship'],
        'Trainer Aircraft': ['trainer aircraft', 'trainer/light aircraft', 'light/trainer aircraft', 'target tow aircraft'],
        'Transport Aircraft': ['transport aircraft', 'light transport aircraft', 'tanker/transport aircraft', 'transport ac/helicopter', 'transport/trainer aircraft']
    },
    # Helicopter Sub-categories
    'Helicopters': {
        'Combat Helicopters': ['combat helicopter', 'ASW helicopter', 'anti-ship helicopter', 'SIGINT helicopter'],
        'Transport/Utility Helicopters': ['transport helicopter', 'light helicopter'],
        'Specialized Helicopters': ['AEW helicopter', 'ASW Helicopter', 'helicopter']
    },
    # Unmanned Vehicles Sub-categories
    'Unmanned Vehicles': {
        'UAV/Drone': ['UAV', 'armed UAV', 'light aircraft/UAV', 'loitering munition'],
        'Reconnaissance/Surveillance UAV': ['reconnaissance AV', 'maritime patrol UAV']
    },
    'Naval Vessels' :{
        'Combat Vessel': ['frigate', 'destroyer', 'corvette', 'submarine', 'gunboat', 'cruiser', 'aircraft carrier', 
                         'corvette/minesweeper', 'FAC (Fast Attack Craft)', 'minesweeper', 'nuclear submarine', 'SSB (Ballistic Missile Submarine)',
                         'midget submarine', 'FAC', 'SSB'],
        'Support Vessel' : [ 'patrol craft', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'landing ship', 'salvage ship', 
        'survey ship', 'minelayer', 'replenishment tanker', 'OPV/transport ship', 'support/landing ship', 
        'landing/patrol craft', 'transport ship', 'landing craft', 'tug', 'OPV/tug', 'frigate/landing ship', 'SIGINT ship', 'patrol craft/transport craft',
        'transport craft', 'support craft', 'cargo craft', 'icebreaker/OPV']
    },
    'Ground Vehicles/Artillery' : {
    'Main Battle Tanks': ['tank', 'light tank', 'training tank', 'tank chassis', 'tank destroyer'],
    'Armored Personnel Carriers (APC)': ['APC', 'APC/APV', 'APC/IFV', 'APC/CP', 'apc (Armored Personnel Carrier)', 'apc'],
    'Infantry Fighting Vehicles (IFV)': ['IFV', 'IFV/AFSV', 'IFV/APC turret', 'IFV turret'],
    'Armored Reconnaissance Vehicles (ARV)': ['ARV', 'AEV/ARV'],
    'Armored Fighting Support Vehicles (AFSV)': ['AFSV', 'AMV', 'ALV (Armored Logistics Vehicle)', 'ALV'],
    'Light Armored Vehicles (APV)': ['APV', 'armoured car'],
    'Self-Propelled Artillery': ['self-propelled gun chassis', 'self-propelled AD system', 'self-propelled gun', 'self-propelled MRL', 'self-propelled AA gun',
                                'AA gun (Anti-Aircraft Gun)', 'self-propelled mortar', 'SPG turret', 'self-propelled AA gun turret'],
    'Engineering Vehicles': ['armoured bridgelayer', 'AEV (Armored Engineering Vehicle)', 'AEV'],
    'Tank Components': ['tank turret'],
    'Artillery' : ['towed gun', 'mortar', 'coastal defence system', 'towed MRL', 'naval gun', 'anti-aircraft gun', 'ASW mortar', 'helicopter turret', 'anti-tank AV turret',
                  'APC turret', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun']
    },
    'Missiles/Rockets/Bombs' : {
    'Surface-to-Air Missiles (SAM)': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile', 'ABM/SAM system', 'ASM (Anti-Ship Missile)', 'ABM system'
    ],
    'Surface-to-Surface Missiles (SSM)': [
        'SSM', 'SSM launcher', 'SSM TEL', 'SSM/ASM', 'SSM/anti-ship missile', 'anti-ship missile/SSM', 'land-attack missile'
    ],
    'Air-to-Air Missiles (AAM)': [
        'BVRAAM', 'SRAAM', 'BVRAAM/SAM'
    ],
    'Anti-Ship Missiles (ASM)': [
        'anti-ship missile', 'anti-ship missile/ASM', 'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'anti-ship/anti-radar missile',
        'naval mine/torpedo', 'anti-ship missile/ASM/SAM', 'ASM'
    ],
    'Anti-Radar Missiles': [
        'anti-radar missile', 'anti-radar missile/ASM'
    ],
    'Anti-Tank Missiles': [
        'anti-tank missile', 'anti-tank missile/ASM'
    ],
    'Guided Bombs and Rockets': [
        'guided bomb', 'guided rocket', 'guided glide bomb', 'guided rocket/SSM', 'guided rocket/ASM', 'guided shell'
    ],
    'Land-Attack Missiles': [
        'anti-ship/land-attack missile', 'land-attack missile'
    ],
    'Anti-Submarine Warfare (ASW) Weapons': [
        'ASW torpedo', 'ASW MRL', 'anti-ship/ASW torpedo', 'ASW missile', 'ASW rocket launcher'
    ],
    'Naval Rocket Launchers': [
        'naval MRL'
    ],
    'Ballistic Missiles' : ['SLBM (Submarine-Launched Ballistic Missile)', 'SLBM']    
    
    },
    'Radar/Detection Systems' : {
    'Air Search Radar': [
        'air search radar', 'AEW radar', 'air search system', 'air/sea search radar', 'air/ground surv radar'
    ],
    'Sea Search Detection Systems': [
        'sea search radar', 'ground/sea search radar', 'ASW sonar', 'SONAR', 'submarine sonar', 'AGS/MP aircraft radar'
    ],
    'Ground Surveillance Radar': [
        'ground surv radar', 'ground/sea search radar', 'air/ground surv radar', 'AGS/SIGINT system'
    ],
    'Multi-function Radar': [
        'multi-function radar', 'multi-role radar'
    ],
    'Fire Control Radar': [
        'fire control radar', 'SAM system radar', 'AGS radar', 'EO system', 'EO search/fire control', 'Naval EO system'
    ],
    'Artillery/Weapon Locating Radar': [
        'artillery locating radar'
    ],
    'Aircraft-Mounted Systems': [
        'MP aircraft radar', 'aircraft EO system', 'aircraft EO/radar system', 'combat aircraft radar', 'combat heli radar',
        'aircraft recce system'
    ],
    'Other Detection Systems': ['height-finding radar', 'recce satellite', 'surveillance satellite'
                               ]
    },
    'Specialized Equipment' : {
    'Aircraft Engines': [
        'aircraft engine', 'turbofan', 'turbojet', 'turboprop'
    ],
    'Vehicle Engines': [
        'vehicle engine', 'VEHICLE ENGINE', 'gas turbine', 'AIP engine', 'nuclear reactor', 'ship engine'
    ],
    'Refueling and Logistics Systems': [
        'air refuel system', 'AALS'
    ],
    'Intelligence and Surveillance Systems': [
        'SIGINT system', 'SIGINT aircraft', 'maritime patrol aircraft'
    ]
}
    # Add more sub-categories as needed in similar structure...
}

# The mapping functions will work the same, now with the expanded mappings.

def map_category(description):
    for category, items in category_mapping.items():
        if description in items:
            return category
    return 'Unknown'

def map_sub_category(description):
    for category, sub_categories in sub_category_mapping.items():
        for sub_category, items in sub_categories.items():
            if description in items:
                return sub_category
    return 'Unknown'

# Applying the mapping functions
df['Category'] = df['Weapon description'].apply(map_category)
df['Sub-Category'] = df['Weapon description'].apply(map_sub_category)



In [7]:
# Creating a DataFrame with three columns: 'Description', 'Category', and 'Sub-category' for exporting to csv for sharing 
csv_mapping_data = {'Category': [], 'Sub-category': [], 'Description': []}

for category, subcategories in sub_category_mapping.items():
    for sub_category, descriptions in subcategories.items():
        for description in descriptions:
            csv_mapping_data['Category'].append(category)
            csv_mapping_data['Sub-category'].append(sub_category)
            csv_mapping_data['Description'].append(description)

# Generating the DataFrame
df_three_columns = pd.DataFrame(csv_mapping_data)
df_three_columns.head()

,Category,Sub-category,Description
0,Aircraft,Fighter/Combat Aircraft,fighter aircraft
1,Aircraft,Fighter/Combat Aircraft,trainer/combat aircraft
2,Aircraft,Fighter/Combat Aircraft,FGA aircraft
3,Aircraft,Fighter/Combat Aircraft,FGA/EW aircraft
4,Aircraft,Fighter/Combat Aircraft,ground attack aircraft


In [8]:
df_three_columns.to_csv('final_data/description_mapping.csv')

In [9]:
df[df['Sub-Category']=='Unknown']['Weapon description'].unique()

array([nan], dtype=object)

In [10]:
# drop nonsense columns
df.drop([' .1', ' .2', ' '], axis=1, inplace=True)
df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0,Aircraft,Fighter/Combat Aircraft
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0,Specialized Equipment,Aircraft Engines
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6,Specialized Equipment,Aircraft Engines
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM)
4,Afghanistan,United Kingdom,2009.0,2.0,Mi-17,transport helicopter,2.0,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8,Helicopters,Transport/Utility Helicopters


In [11]:
filtered_df = df[df['Year(s) of delivery'].str.len() > 4]
print(filtered_df['Year(s) of delivery'].unique())



['1977; 1978' '1988; 1989; 1990; 1991' '1980; 1981; 1982' ...
 '1968; 1969; 1970; 1971; 1972; 1973; 1974; 1975; 1976; 1977; 1978; 1979; 1980; 1981; 1982'
 '1963; 1964; 1965; 1966; 1967; 1968; 1969; 1970; 1971; 1972; 1973; 1974; 1975; 1976; 1977; 1978; 1979; 1980; 1981; 1982; 1983; 1984; 1985'
 '1964; 1965; 1966; 1967; 1968; 1969; 1970; 1971; 1972; 1973; 1974; 1975; 1976; 1977; 1978; 1979; 1980; 1981; 1982; 1983']


In [12]:
def count_years_delivered(years_of_delivery):
    # Count semicolons and return one less than the count to count number of unique years weapons delivered in
    if isinstance(years_of_delivery, str):
        # Count semicolons if the entry is a string
        return years_of_delivery.count(';')+1
    else:
        # Return 0 for non-string entries (like NaN)
        return 0

In [13]:
df['Num years Delivered In'] = df['Year(s) of delivery'].apply(count_years_delivered)
df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category,Num years Delivered In
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0,Aircraft,Fighter/Combat Aircraft,1
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0,Specialized Equipment,Aircraft Engines,1
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6,Specialized Equipment,Aircraft Engines,2
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM),4
4,Afghanistan,United Kingdom,2009.0,2.0,Mi-17,transport helicopter,2.0,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8,Helicopters,Transport/Utility Helicopters,1


In [14]:
# Get first and last year of delivery in seperate cols

In [15]:
def get_first_year(years_of_delivery):
    if isinstance(years_of_delivery, str):
        # Split the string by semicolons and get the first year, stripping any spaces
        return years_of_delivery.split(';')[0].strip()
    else:
        # Return NaN for non-string entries (like NaN)
        return float('nan')

def get_last_year(years_of_delivery):
    if isinstance(years_of_delivery, str):
        # Split the string by semicolons and get the last year, stripping any spaces
        return years_of_delivery.split(';')[-1].strip()
    else:
        # Return NaN for non-string entries (like NaN)
        return float('nan')

In [16]:
df['First Year'] = df['Year(s) of delivery'].apply(get_first_year)
df['Last Year'] = df['Year(s) of delivery'].apply(get_last_year)
df.head(10)

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category,Num years Delivered In,First Year,Last Year
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0,Aircraft,Fighter/Combat Aircraft,1,2018,2018
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0,Specialized Equipment,Aircraft Engines,1,2005,2005
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6,Specialized Equipment,Aircraft Engines,2,1977,1978
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM),4,1988,1991
4,Afghanistan,United Kingdom,2009.0,2.0,Mi-17,transport helicopter,2.0,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8,Helicopters,Transport/Utility Helicopters,1,2010,2010
5,Afghanistan,Soviet Union,1980.0,1000.0,Fleyta/Skorpion,anti-tank missile,1000.0,1980; 1981; 1982,New,For Mi-24A helicopters,0.02,20.0,20.0,Missiles/Rockets/Bombs,Anti-Tank Missiles,3,1980,1982
6,Afghanistan,Soviet Union,1988.0,250.0,R-60,SRAAM,250.0,1988; 1989; 1990,New,For MiG-23MF and MiG-21bis combat aircraft,0.09,22.5,22.5,Missiles/Rockets/Bombs,Air-to-Air Missiles (AAM),3,1988,1990
7,Afghanistan,Soviet Union,1987.0,40.0,MiG-21MF,fighter aircraft,40.0,1987; 1988; 1989; 1990,Second hand,Second-hand; probably incl some MiG-21UM,4.28,171.2,171.2,Aircraft,Fighter/Combat Aircraft,4,1987,1990
8,Afghanistan,Soviet Union,1985.0,100.0,Strela-3,portable SAM,100.0,1986; 1987; 1988; 1989,New,Probably incl for Mi-24 combat helicopters,0.06,6.0,6.0,Missiles/Rockets/Bombs,Surface-to-Air Missiles (SAM),4,1986,1989
9,Afghanistan,Soviet Union,1979.0,50.0,Mi-8T,transport helicopter,50.0,1979; 1980; 1981,New,NaN,6.00,300.0,300.0,Helicopters,Transport/Utility Helicopters,3,1979,1981


In [17]:
df.to_csv('final_data/cleaned_transfer_data.csv')

## Create csv with one row per year delivered

In [18]:
delivered_df = df.dropna(subset=['First Year'])

delivered_df['First Year'] = delivered_df['First Year'].astype(int)

# Expand rows based on `Num years Delivered In`
annual_delivery_df = delivered_df.loc[delivered_df.index.repeat(delivered_df['Num years Delivered In'])].copy()

# Calculate the `year_delivered` for each expanded row
annual_delivery_df['year_delivered'] = annual_delivery_df.groupby(level=0).cumcount() + annual_delivery_df['First Year']
annual_delivery_df['num_delivered_in_year'] = annual_delivery_df['Number delivered']/annual_delivery_df['Num years Delivered In']
annual_delivery_df['SIPRI_TIV_delivered_in_year'] = annual_delivery_df['num_delivered_in_year']*annual_delivery_df['SIPRI TIV per unit']


/var/folders/63/jzv4h4hj37j3wl3_r5y5828h0000gn/T/ipykernel_4027/3196642239.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivered_df['First Year'] = delivered_df['First Year'].astype(int)


In [19]:
annual_delivery_df = annual_delivery_df.rename(columns={'Number delivered':'Total number delivered'})
annual_delivery_df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Total number delivered,Year(s) of delivery,status,Comments,...,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category,Num years Delivered In,First Year,Last Year,year_delivered,num_delivered_in_year,SIPRI_TIV_delivered_in_year
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,...,27.0,27.0,Aircraft,Fighter/Combat Aircraft,1,2018,2018,2018,6.0,27.00
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,...,3.0,3.0,Specialized Equipment,Aircraft Engines,1,2005,2005,2005,6.0,3.00
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,...,15.6,15.6,Specialized Equipment,Aircraft Engines,2,1977,1978,1977,13.0,7.80
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,...,15.6,15.6,Specialized Equipment,Aircraft Engines,2,1977,1978,1978,13.0,7.80
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,...,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM),4,1988,1991,1988,575.0,718.75


In [20]:
annual_delivery_df.to_csv('final_data/annual_delivery_data.csv')